In [48]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

# ✅ 설정
기준일 = datetime(2025, 4, 28, tzinfo=ZoneInfo("Asia/Seoul"))
df_index = pd.read_csv("./index_list.csv")

# ✅ 날짜 계산
start_date = 기준일 - timedelta(days=370)
end_date = 기준일 + timedelta(days=1)

all_dates = pd.date_range(start=start_date, end=기준일)
business_days = all_dates[all_dates.weekday < 5]

last_year = 기준일.year - 1
last_year_end = max([d for d in business_days if d.year == last_year])

prev_month = 기준일.month - 1 if 기준일.month > 1 else 12
prev_month_year = 기준일.year if 기준일.month > 1 else 기준일.year - 1
prev_month_end = datetime(prev_month_year, prev_month, 1, tzinfo=ZoneInfo("Asia/Seoul")) + timedelta(days=31)
prev_month_end = prev_month_end.replace(day=1) - timedelta(days=1)
prev_month_end = max([d for d in business_days if d.month == prev_month and d <= 기준일])

recent_business_days = [d for d in business_days if d < 기준일][-5:]

weekday_map = {0: "월", 1: "화", 2: "수", 3: "목", 4: "금", 5: "토", 6: "일"}
headers = [
    f"{last_year % 100}년 未",
    f"{기준일.year % 100}.{prev_month}월",
]
headers += [f"{d.month}/{d.day}({weekday_map[d.weekday()]})" for d in recent_business_days]
headers += ["변동량", "변동률(%)"]  # ✅ 변동률(%)로 변경

# ✅ 결과 저장용
records = []
raw_records = []

# ✅ 데이터 수집
for _, row in df_index.iterrows():
    ticker = row["티커"]

    try:
        data = yf.download(
            ticker,
            start=start_date.strftime("%Y-%m-%d"),
            end=end_date.strftime("%Y-%m-%d"),
            progress=False
        )

        if data.empty:
            print(f"❗ 다운로드 실패: {ticker}")
            continue

        if 'Close' not in data.columns:
            print(f"❗ 'Close' 없음: {ticker}")
            continue

        # ✅ 인덱스를 풀고 Date 컬럼으로
        data = data.reset_index()

        # ✅ 필요한 컬럼만 추림
        data = data[["Date", "Close", "Open", "High", "Low", "Volume"]].copy()

        # ✅ 필요한 열 추가
        data["국가"] = row["국가"]
        data["구분"] = row["구분"]
        data["단위"] = row["항목명_짧은"]
        data["Ticker"] = row["티커"]

        # ✅ 순서 재정렬
        data = data[["국가", "구분", "단위", "Ticker", "Date", "Close", "Open", "High", "Low", "Volume"]]

        # ✅ 헤더 강제 재정의
        data.columns = ["국가", "구분", "단위", "Ticker", "Date", "Close", "Open", "High", "Low", "Volume"]

        raw_records.append(data)

        # ✅ 요약 테이블 생성
        price_series = data.set_index("Date")["Close"].dropna()

        record = {
            "국가": row["국가"],
            "구분": row["구분"],
            "단위": row["항목명_짧은"],
        }

        values = {}
        dates_needed = [last_year_end, prev_month_end] + recent_business_days
        for d, h in zip(dates_needed, headers[:-2]):
            nearest_date_candidates = [dt for dt in price_series.index if dt.date() <= d.date()]
            if not nearest_date_candidates:
                values[h] = None
            else:
                nearest_date = max(nearest_date_candidates)
                value = price_series.loc[nearest_date]
                if isinstance(value, pd.Series):
                    value = value.item()
                values[h] = value

        # ✅ 변동량과 변동률(%) 계산 + 예외처리 (하이픈)
        try:
            day_1 = values[headers[6]]  # 1일전
            day_2 = values[headers[5]]  # 2일전

            if (day_1 is None) or (day_2 is None) or (day_2 == 0):
                변동량 = "-"
                변동률 = "-"
            else:
                변동량 = day_1 - day_2
                변동률 = (변동량 / day_2) * 100
        except:
            변동량 = "-"
            변동률 = "-"

        values["변동량"] = 변동량
        values["변동률(%)"] = 변동률

        record.update(values)
        records.append(record)

    except Exception as e:
        print(f"❗ 에러 발생 {ticker}: {e}")
        continue

# ✅ 원본 테이블 만들기
df_raw = pd.concat(raw_records, axis=0, ignore_index=True)

# ✅ 요약 테이블 만들기
df_final = pd.DataFrame(records)

# ✅ 국가, 구분 소팅 추가
country_order = {"베트남": 0, "인니": 1, "한국": 2, "필리핀": 3, "중국": 4}
category_order = {"지표": 0, "환율": 1}

df_final["국가순서"] = df_final["국가"].map(country_order)
df_final["구분순서"] = df_final["구분"].map(category_order)

df_final = df_final.sort_values(["국가순서", "구분순서"]).drop(columns=["국가순서", "구분순서"]).reset_index(drop=True)

# ✅ 출력
print("\n✅ 요약 테이블 (df_final)")
print(df_final)

print("\n✅ 원본 데이터 (df_raw)")
print(df_raw)

# ✅ CSV 저장 (필요시)
# df_final.to_csv("summary_result.csv", index=False, encoding="utf-8-sig")
# df_raw.to_csv("raw_result.csv", index=False, encoding="utf-8-sig")

# ✅ 주말제외 5일전 ~ 주말제외 1일전 날짜 범위 프린트
start_day = recent_business_days[0]
end_day = recent_business_days[-1]
print(f"\n📅 기간: {start_day.month}/{start_day.day}~{end_day.month}/{end_day.day}")



✅ 요약 테이블 (df_final)
    국가  구분        단위         24년 未         25.3월       4/21(월)       4/22(화)  \
0  베트남  지표  VN Index   1266.780029   1306.859985   1207.069946   1197.130005   
1  베트남  환율   USD/VND  25476.000000  25550.000000  25830.000000  25871.000000   
2   인니  지표       IDX   7079.904785   6510.620117   6445.966797   6538.266113   
3   인니  환율   USD/IDR  16085.599609  16575.000000  16794.599609  16814.900391   
4   한국  지표     KOSPI   2399.489990   2481.120117   2488.419922   2486.639893   
5   한국  환율   USD/KRW   1467.390015   1469.410034   1414.229980   1421.640015   
6  필리핀  지표      PSEi   6528.790039   6180.720215   6138.000000   6145.589844   
7  필리핀  환율   USD/PHP     58.029999     57.369999     56.544998     56.476002   
8   중국  지표      SEZE  10414.610352  10504.330078   9905.530273   9870.049805   
9   중국  환율   USD/CNY      7.298100      7.262800      7.311900      7.299000   

        4/23(수)       4/24(목)       4/25(금)         변동량    변동률(%)  
0   1211.000000   1223.349976 

In [65]:
import shutil
from openpyxl import load_workbook

# ✅ 템플릿 파일 경로
template_path = "./tmp/국가별 주가환율 테이블 템플릿_edit_v2.xlsx"

# ✅ 새로운 파일 이름 (기준일 기반 yymmdd)
save_date = 기준일.strftime("%y%m%d")
save_path = f"./result/국가별 주가환율 정보_{save_date}.xlsx"

# ✅ 템플릿 복사
shutil.copy(template_path, save_path)

# ✅ 엑셀 파일 열기
wb = load_workbook(save_path)

# ✅ df_final 저장 (table 시트)
ws_table = wb["table"]

# ✅ 먼저 기존 값 Clear (값만 지우고, 서식/테이블 구조 유지)
for row in ws_table.iter_rows(min_row=1, max_row=ws_table.max_row, min_col=1, max_col=ws_table.max_column):
    for cell in row:
        cell.value = None

# ✅ df_final 값 다시 채워넣기
for idx, col_name in enumerate(df_final.columns, 1):
    ws_table.cell(row=1, column=idx).value = col_name

for row_idx, row in enumerate(df_final.values, 2):
    for col_idx, value in enumerate(row, 1):
        ws_table.cell(row=row_idx, column=col_idx).value = value

# ✅ name 시트 A1 입력
ws_name = wb["name"]
ws_name["A1"] = f"{start_day.month}/{start_day.day}~{end_day.month}/{end_day.day}"

# ✅ df_raw 저장 (rawdata 시트)
ws_raw = wb["rawdata"]
for idx, col_name in enumerate(df_raw.columns, 1):
    ws_raw.cell(row=1, column=idx).value = col_name
for row_idx, row in enumerate(df_raw.values, 2):
    for col_idx, value in enumerate(row, 1):
        ws_raw.cell(row=row_idx, column=col_idx).value = value

# ✅ 파일 저장
wb.save(save_path)

print(f"\n✅ 엑셀 파일 저장 완료: {save_path}")



✅ 엑셀 파일 저장 완료: ./result/국가별 주가환율 정보_250428.xlsx
